# 향후 판매량 예측 경진대회 이해

<div>
    <img src="attachment:349bc070-ccb4-4ea3-8c2c-c50df711119a.png" width="1000"/>
</div>

### 학습 목표
- 다양한 피처 엔지니어링 기법을 배워봅시다!

### 학습 키워드
- 유형 및 평가지표: 회귀, RMSE
- 탐색적 데이터 분석: 데이터 병합, 그룹화
- 머신러닝 모델: LightGBM
- 피처 엔지니어링: 피처명 한글화, 데이터 다운캐스팅, 조합, 이어 붙이기, 병합, 이상치 제거, 파생 피처 생성
<div>
    <img src="attachment:08682f1e-9781-441f-a927-9df0959867f5.png" width="700"/>
</div>

### 경진대회 Overview
- 과거 판매 데이터를 토대로 향후 판매량 예측
 - 타깃값은 판매량이므로 범주형 데이터가 아님. 따라 회귀 문제에 속함
- 주어진 데이터는 2013년 1월부터 2015년 10월까지의 일별 판매 내역
 - 더불어 상점, 상품, 상품분류에 관한 추가 데이터도 있음
 - 이 데이터를 기반으로 2015년 11월 각 상점의 상품별 월간 판매량을 예측해야 함
 - 피처: 상점 및 상품에 관한 정보, 타깃값: 월간 판매량
- 주의점
 - 상품별 월간 판매량(타깃값)은 0개에서 20개 사이여야 함
 - 타깃값뿐만 아니라 판매량과 관련된 피처는 모두 0~20 사이로 값을 제한해야 함
 <div>
    <img src="attachment:dfc0ad5b-485a-4f0e-bfc7-93ed592aa64c.png" width="700"/>
</div>

# 9.2 향후 판매량 예측 경진대회 탐색적 데이터 분석

## 9.2.1 데이터 둘러보기

In [ ]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/competitive-data-science-predict-future-sales/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### sales_train 데이터

In [ ]:
# 예측해야 되는 것은 월간판매이기 때문.
# 러시아 화페단위 루블.
# 상점별 일간 판매량 1행당. 판매량이 -1은 오류값 왜 그런지 모름.
# 일별이기 때문에 월별로 바꿔 줘야함. 그래서 타깃값을 date_block_num을 기준으로 gropby함.
# 가장 최근 데이터인 검증데이터로 사용한다. 기간은 상관없음. 다만 검증데이터가 너무 많아지면
# 훈련데이터가 줄어들기 때문에 좋지 않음. 훈련 정확도 vs 검증 신뢰도. 데이터가 큰경우에는 한
# 시계열 데이터가 아닌경우에는 아까 교차검증대로 함.
# 미래를 바탕으로 과거를 예측하는 것은 말이 안됨. 과거를 기반으로 미래를 예측해야됨.
# 그래서 시계열 데이터는 시간의 흐름이 중요함. 시간의 흐름에 따라서 뒤쪽에 있는 것을 검증데이터를 한다.
# 1. date 피처 제거
# date_block_num은 편의상 사용하는 날짜(월) 구분자: 0은 2013년 1월, 1은 2013년 2월, 33은 2015년 10월
# 최종적으로 월별 판매량을 구해야 하니 ‘월’ 구분자(date_block_num)만 있으면 됨, 따라서 date 피처 삭제
# [분석 결과] 월별 판매량만 구하면 되니 date 피처 제거
# item_cnt_day 피처는 당일 판매량
# 타깃값은 ‘월간’ 판매량이므로, date_block_num 피처를 기준으로 그룹화해서 item_cnt_day 값을 합하면 타깃값이 됨
# [분석 결과] 타깃값 = date_block_num 피처의 값이 같은 데이터들의 item_cnt_day 값의 합
# 본 데이터는 시계열 데이터이므로 시간 흐름 자체가 중요한 정보
# 그렇기 때문에 2013년 1월 ~ 2015년 9월까지 판매 내역을 훈련 데이터, 2015년 10월 판매 내역을 검증 데이터로 사용
# [분석 결과] 훈련 데이터 중 가장 최근인 2015년 10월 판매 내역을 검증 데이터로 사용
# sales_train
sales_train

- date_block_num은 편의상 사용하는 날짜(월) 구분자: 0은 2013년 1월, 1은 2013년 2월, 33은 2015년 10월
- 최종적으로 월별 판매량을 구해야 하니 ‘월’ 구분자(date_block_num)만 있으면 됨, 따라서 date 피처 삭제

#### **<font color='orange'>[분석 결과] 월별 판매량만 구하면 되니 date 피처 제거</font>**

- item_cnt_day 피처는 당일 판매량
- 타깃값은 ‘월간’ 판매량이므로, date_block_num 피처를 기준으로 그룹화해서 item_cnt_day 값을 합하면 타깃값이 됨

#### **<font color='orange'>[분석 결과] 타깃값 = date_block_num 피처의 값이 같은 데이터들의 item_cnt_day 값의 합</font>**


- 본 데이터는 시계열 데이터이므로 시간 흐름 자체가 중요한 정보
- 그렇기 때문에 2013년 1월 ~ 2015년 9월까지 판매 내역을 훈련 데이터, 2015년 10월 판매 내역을 검증 데이터로 사용

#### **<font color='orange'>[분석 결과] 훈련 데이터 중 가장 최근인 2015년 10월 판매 내역을 검증 데이터로 사용</font>**

In [ ]:
# 메모리 사용량을 가능한 한 줄여주는 게 바람직.
# [분석 결과] 메모리 관리 필요. 메모리 관리할 수록 속도가 빨라짐.
sales_train.info(show_counts=True)

- 메모리 사용량을 가능한 한 줄여주는 게 바람직

#### **<font color='orange'>[분석 결과] 메모리 관리 필요 </font>**

### shops 데이터

In [ ]:
sales_train.head(2)

In [ ]:
shops.head()

- 상점명(shop_name)은 러시아어로 기재됨, 상점명의 첫 단어는 상점이 있는 도시를 나타냄
- 상점 ID(shop_id)는 sales_train에도 있는 피처, 그러므로 shop_id를 기준으로 sales_train과 shops를 병합

#### **<font color='orange'>[분석 결과] 상점 이름의 첫 단어는 도시를 뜻함 </font>**

#### **<font color='orange'>[분석 결과] shop_id를 기준으로 sales_train과 shops 병합 </font>**

In [ ]:
shops.info()

- 상점은 딱 60개만 있음
- 결측값도 없고, 데이터 개수가 적어서 메모리 사용량도 1.1KB 정도로 적음

### items 데이터

In [ ]:
items.head()

- 상품명(러시아어), 상품 ID, 상품분류 ID로 구성
- 상품명에서는 유용한 정보를 얻기 어려워 모델링 시 제거
- item_id 피처는 sales_train에도 존재, 따라서 item_id 피처를 기준으로 sales_train과 items 병합 가능

#### **<font color='orange'>[분석 결과] 상품명 피처 제거 </font>**

#### **<font color='orange'>[분석 결과] item_id를 기준으로 sales_train과 items 병합 </font>**

In [ ]:
items.info()

### item_categories 데이터

In [ ]:
sales_train.head(2)

In [ ]:
# 상점명(shop_name)은 러시아어로 기재됨, 상점명의 첫 단어는 상점이 있는 도시를 나타냄
# 상점 ID(shop_id)는 sales_train에도 있는 피처, 그러므로 shop_id를 기준으로 sales_train과 shops를 병합
item_categories.head()

- 상품분류명과 상품분류 ID로 구성
- sales_train에도 item_category_id 피처가 있어서, 이 피처를 기준으로 sales_train과 item_categories 병합
- 상품분류명의 첫 단어는 대분류를 뜻함

#### **<font color='orange'>[분석 결과] item_category_id를 기준으로 sales_train과 item_categories 병합 </font>**

#### **<font color='orange'>[분석 결과] 상품분류명에서 첫 단어는 대분류 </font>**

In [ ]:
item_categories.info()

#### **<font color='orange'>[분석 결과] 모든 데이터에 결측값 없음 </font>**

### 테스트 데이터

In [ ]:
test

### 제출 샘플 데이터

In [ ]:
# 제출 샘플 데이터.
# 1번쩨 아이디에서의 월간 판매율은 몇이냐.
submission.head()

### 데이터 병합
- 피처 요약표를 만들고 시각화를 하기 위해 데이터 병합
 - 앞서 설명한 바와 같이 특정 피처를 기준으로 병합
 - 판다스 merge( ) 함수로 구현
- sales_train, shops, items, item_categories를 모두 병합해 그 결과를 train에 할당

1. 왼쪽 테이블 기준으로 병합

![image.png](attachment:78c2b1d4-e68f-44c9-b8b6-4ed98285605d.png)

2. 오른쪽 테이블 기준으로 병합

![image.png](attachment:3304a91f-a371-4bc3-b247-9a33a2612a21.png)

In [ ]:
# 별도의 csv파일들을 기준을 잡아서 한 훈련 데이터로 합친것.
train = sales_train.merge(shops, on='shop_id', how='left')
train = train.merge(items, on='item_id', how='left')
train = train.merge(item_categories, on='item_category_id', how='left')

train.head()

### 피처 요약표 만들기
- 데이터 타입, 결측값 개수, 고윳값 개수, 첫 번째 값, 두 번째 값을 포함한 피처 요약표 만들기(p.622)


In [ ]:
def resumetable(df):
    print(f'데이터 세트 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': '피처'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고윳값 개수'] = df.nunique().values
    summary['첫 번째 값'] = df.loc[0].values
    summary['두 번째 값'] = df.loc[1].values

    return summary

In [ ]:
resumetable(train)

- shop_id와 shop_name, item_id와 item_name, item_category_id와 item_category_name 피처의 고윳값 개수가 서로 같음

#### **<font color='orange'>[분석 결과] 상점ID, 상품ID, 상품분류ID는 각각 상점명, 상품명, 상품분류명과 1:1 매칭되므로 둘 중 하나 제거 </font>**

## 9.2.2 데이터 시각화
- 병합한 train을 이용해 데이터 시각화
- 피처 개수가 많지 않고, 그중 일부는 식별자거나 문자 데이터라서 시각화할 게 별로 없음

### 일별 판매량
- train에서 식별자나 문자 데이터를 빼면 item_cnt_day 피처와 item_price 피처만 남음
- 우선 item_cnt_day 피처 시각화

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

sns.boxplot(y='item_cnt_day', data=train);

- 이상치가 많아서 박스플롯 모양이 이상함
- 과한 이상치는 제거해야 함
- 여기선 item_cnt_day 1,000 이상인 데이터 제거

#### **<font color='orange'>[분석 결과] 일별 판매량 1,000 이상인 데이터(이상치) 제거 </font>**

### 상품 가격

In [ ]:
sns.boxplot(y='item_price', data=train);

- 300,000루블(약 450만 원)이 넘는 판매가 때문에 박스플롯 모양이 역시 납작함
- 추후 판매가 50,000루블 이상인 이상치는 제거

#### **<font color='orange'>[분석 결과] 판매가 50,000 이상인 데이터(이상치) 제거 </font>**

### 그룹화
- 특정 피처를 기준으로 그룹화해서 시각화
 - 다음 코드는 train의 date_block_num 피처를 기준으로 그룹화해 item_cnt_day 피처 값의 합(sum)을 구하는 코드 (월별(date_block_num) 월간 판매량(item_cnt_day의 합)을 구한다는 말)

In [ ]:
train.head(2)

In [ ]:
group = train.groupby('date_block_num').agg({'item_cnt_day': 'sum'})
group.reset_index() # 인덱스 재설정

- reset_index( )를 호출하지 않으면 그룹화한 date_block_num 피처가 인덱스로 설정됨
- 그래프를 그릴 때 date_block_num 피처를 사용해야 해서 인덱스를 재설정함

#### groupby 원리
- 한 개 이상 피처를 기준으로 데이터 분리 (앞 코드에서 기준이 되는 피처는 date_block_num)
- 분리된 각 그룹에서 함수를 적용해 집곗값 구함 (앞 코드에서는 agg( ) 메서드로 item_cnt_day 피처에 ‘sum’ 함수를 적용해 판매량 합계를 구함)
- 기준 피처별로 집곗값 결과를 하나로 결합
 <div>
    <img src="attachment:52ea4feb-cc6d-4b31-b544-d64c61566b83.png" width="700"/>
</div>

### 월별 판매량
- groupby 합 연산 결과인 group을 활용해 시각화 (= 월별 판매량)

In [ ]:
mpl.rc('font', size=13)
figure, ax = plt.subplots()
figure.set_size_inches(11, 5)

# 월별 총 상품 판매량
group_month_sum = train.groupby('date_block_num').agg({'item_cnt_day': 'sum'})
group_month_sum = group_month_sum.reset_index()

# 월별 총 상품 판매량 막대그래프
sns.barplot(x='date_block_num', y='item_cnt_day', data=group_month_sum)
# 그래프 제목, x축 라벨, y축 라벨명 설정
ax.set(title='Distribution of monthly item counts by date block number',
       xlabel='Date block number',
       ylabel='Monthly item counts');

### 상품분류별 판매량
- 월간 판매량 10,000개 초과인 상품분류만 추출해서 시각화

In [ ]:
figure, ax= plt.subplots()
figure.set_size_inches(11, 5)

# 상품분류별 총 상품 판매량
group_cat_sum = train.groupby('item_category_id').agg({'item_cnt_day': 'sum'})
group_cat_sum = group_cat_sum.reset_index()

# 총 판매량이 10,000개를 초과하는 상품분류만 추출
group_cat_sum = group_cat_sum[group_cat_sum['item_cnt_day'] > 10000]

# 상품분류별 총 상품 판매량 막대그래프
sns.barplot(x='item_category_id', y='item_cnt_day', data=group_cat_sum)
ax.set(title='Distribution of total item counts by item category id',
       xlabel='Item category ID',
       ylabel='Total item counts')
ax.tick_params(axis='x', labelrotation=90) # x축 라벨 회전

### 상점별 판매량
- 월간 판매량 10,000개 초과인 상점만 추출해서 시각화

In [ ]:
figure, ax= plt.subplots()
figure.set_size_inches(11, 5)

# 상점별 총 상품 판매량
group_shop_sum = train.groupby('shop_id').agg({'item_cnt_day': 'sum'})
group_shop_sum = group_shop_sum.reset_index()

group_shop_sum = group_shop_sum[group_shop_sum['item_cnt_day'] > 10000]

# 상점별 총 상품 판매량 막대그래프
sns.barplot(x='shop_id', y='item_cnt_day', data=group_shop_sum)
ax.set(title='Distribution of total item counts by shop id',
       xlabel='Shop ID',
       ylabel='Total item counts')
ax.tick_params(axis='x', labelrotation=90)

### 분석 결과
- [분석 결과] 월별 판매량만 구하면 되니 date 피처 제거
- [분석 결과] 타깃값 = date_block_num 피처의 값이 같은 데이터들의 item_cnt_day 값의 합
- [분석 결과] 훈련 데이터 중 가장 최근인 2015년 10월 판매 내역을 검증 데이터로 사용
- [분석 결과] 메모리 관리 필요
- [분석 결과] 상점 이름의 첫 단어는 도시를 뜻함
- [분석 결과] shop_id를 기준으로 sales_train과 shops 병합
- [분석 결과] 상품명 피처 제거
- [분석 결과] item_id를 기준으로 sales_train과 items 병합
- [분석 결과] item_category_id를 기준으로 sales_train과 item_categories 병합
- [분석 결과] 상품분류명에서 첫 단어는 대분류
- [분석 결과] 모든 데이터에 결측값 없음
- [분석 결과] 상점ID, 상품ID, 상품분류ID는 각각 상점명, 상품명, 상품분류명과 1:1 매칭되므로 둘 중 하나 제거
- [분석 결과] 일별 판매량 1,000 이상인 데이터(이상치) 제거

### 분석 정리
1. 판매량 관련 피처 값은 모두 0~20 사이로 제한해야 함
2. 시계열 데이터이므로 데이터 순서를 꼭 지켜야 함(검증 데이터는 최근 1개월치 이용)
3. 타깃값 : 같은 달 일별 판매량을 합쳐 타깃값(월별 판매량)을 구해야 함
4. 데이터 병합 : 추가 정보 파일(상점, 상품, 상품분류)은 각각의 ID(상점ID, 상품ID, 상품분류ID)를 기준으로 훈련 데이터에 병합할 수 있음
5. 다양한 피처 엔지니어링 후 데이터 크기가 커서 메모리 관리 필요
6. 파생 피처 추가 : 상점명과 상품분류명의 첫 단어는 각각 도시와 대분류
7. 피처 제거 : 월별 판매량만 구하면 되니 date 피처 필요 없음
8. 피처 제거 : 상점ID, 상품ID, 상품분류ID는 각각 상점명, 상품명, 상품분류명과 1:1로 매칭되므로 둘 중 하나만 있으면 됨
9. 이상치 제거 : 일별 판매량과 판매가에는 값이 이상치가 있어서 제거 필요
10. 모든 데이터에서 결측값이 없음